## Overview of the Style Transfer Notebook

This Jupyter Notebook is designed to facilitate interactive style transfer for images and videos using pre-trained deep learning models. The notebook allows users to upload media, select specific models for style transfer, and view the results directly within the interface.

### Key Features:
- **Media Upload Widgets**: Users can upload images and videos to predefined directories using intuitive file upload widgets.
- **Style Transfer Processing**: Implements style transformation functions that standardize media into a format suitable for neural network processing. This includes resizing, tensor conversion, and normalization.
- **Dynamic Model Selection**: Users can choose from a list of pre-trained models stored in a designated directory, enabling different styles to be applied to the uploaded media.
- **Real-time Processing Feedback**: The process of uploading files, performing style transfer, and saving outputs is logged in real-time, providing transparency and feedback on the operations being performed.
- **Output Visualization**: After processing, the stylized images and videos are displayed directly in the notebook, allowing for immediate review and comparison.

### Technologies Used:
- **PyTorch**: Utilized for building and applying neural network models for style transfer.
- **Torchvision**: Provides image transformations and utilities to facilitate working with image data.
- **IPython Widgets**: Creates an interactive GUI within the Jupyter environment, enhancing user engagement and ease of use.
- **skvideo.io**: Handles video file writing and processing, ensuring high-quality video output.

This notebook is perfect for users who wish to experiment with different style transfers on their media files or developers looking for an easily adaptable template for building more complex media processing workflows.

### Required Datasets

**Pre-trained Style Transfer Models**:

The pre-trained models can be downloaded from the following Google Drive folder. Please download all the files at once to ensure you have the complete set of necessary models for this project.

### How to Download Models
To access and download all the models in one go, please follow these simplified steps:

1. Visit the [Google Drive folder with pre-trained models](https://drive.google.com/drive/folders/1aRD6zakhcDImN2Y54qAT6f4801iLcCLB?usp=sharing).
2. Click on the "Download all" button to download the models as a single compressed (zip) file.
3. Unzip the downloaded file into the `models` directory within your project folder.

### Additional Library Installation
To enhance the functionality of the CTPO environment, you may need to install some libraries not pre-installed but required for this notebook. Follow these steps to install the necessary libraries from the `requirements.txt` file:

#### 1. Create and Activate the Virtual Environment:

- Open your terminal or command prompt within the Jupyter notebook. Navigate via `File -> New -> Terminal`.
- Type `bash` to get access to a shell compatible with the following commands.
- Navigate to the project directory where you want to set up the environment:
    ```
    cd /path/to/your/project/directory
    pwd
    ```
- Execute the following commands to create and activate the virtual environment:
    ```
    python3 -m venv --system-site-packages myvenv
    source myvenv/bin/activate
    pip3 install ipykernel
    python -m ipykernel install --user --name=myvenv --display-name="Python (myvenv)"
    ```

#### 2. Install Required Libraries:

- Before running the following commands, load the "Python (myvenv)" kernel to limit the chances of altering the underlying container. Ensure you are in the directory where the Jupyter Notebook and the `myvenv` directory are located. Use `cd` to change to your project directory and `pwd` to verify your current directory.



In [ ]:
!. ./myvenv/bin/activate; pip install -r requirements.txt

In [1]:
import os
import sys
import random
from PIL import Image
import numpy as np
import torch
import glob
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import save_image
from models import TransformerNet, VGG16
from utils import *
from models import TransformerNet
import tqdm
import warnings
from torch.autograd import Variable
import skvideo.io
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

# Ignore all warnings
warnings.filterwarnings("ignore")

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest 

#### Image Upload Functionality

This section sets up an interactive file upload widget allowing users to upload images to a specific target directory ('input/images'). It checks if the directory exists and creates it if necessary. The upload process is logged in detail, including the start of the upload, file handling, and a confirmation once files are saved.


In [3]:
target_directory = 'input/images'
 
# Ensure the target directory exists, create it if it does not
if not os.path.exists(target_directory):
    os.makedirs(target_directory)  # This will create the directory and any necessary parent directories
    print(f"Directory {target_directory} created.")
else:
    print(f"Target directory: {target_directory}")
 
# Function to handle uploaded files
def handle_upload(change):
    print("Upload started...")
    # Print the structure of 'change' to understand its content
    print(change)
    
    for file_upload in change['new']:
        print(f"Handling file: {file_upload}")
        filepath = os.path.join(target_directory, file_upload['name'])
        print(f"Saving to: {filepath}")
        with open(filepath, 'wb') as f:
            f.write(file_upload['content'])
        print(f'Saved {file_upload["name"]} to {filepath}')
    # List the files in the target directory after upload
    print(f'Files in target directory ({target_directory}): {list(Path(target_directory).glob("*"))}')
    print("Upload completed.")
 
# Create an output widget to capture print statements
output = widgets.Output()
 
# Create an upload widget
upload_widget = widgets.FileUpload()
 
# Function to handle the change event using output widget
def handle_upload_with_output(change):
    with output:
        handle_upload(change)
 
# Attach the observer to the upload widget
upload_widget.observe(handle_upload_with_output, names='value')
 
# Display the upload widget and output widget
display(upload_widget, output)
 

Directory input/images created.


FileUpload(value=(), description='Upload')

Output()

#### Style Transformation and Image Processing

Defines a `style_transform` function that standardizes images to a consistent format suitable for style transfer, including resizing, tensor conversion. The `load_model` function loads a pre-trained model, setting it to evaluation mode for inference. 

The `stylize_image` function applies the style transformation model to an input image, performing the style transfer. This function is triggered by an interactive button which allows the user to select an image and a model for style transfer. The processed image is saved and displayed in the notebook.


In [4]:
# Ensure output directory exists
os.makedirs("images/outputs", exist_ok=True)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a function for image transformations
def style_transform():
    return transforms.Compose([
        Resize(512),  # or another size that fits the model
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# Load and prepare the model
def load_model(checkpoint_path):
    transformer = TransformerNet().to(device)
    transformer.load_state_dict(torch.load(checkpoint_path,map_location=device))
    transformer.eval()
    return transformer

# Perform style transfer
def stylize_image(image_path, model):
    image_tensor = Variable(style_transform()(Image.open(image_path))).to(device)
    image_tensor = image_tensor.unsqueeze(0)
    with torch.no_grad():
        stylized_image = denormalize(model(image_tensor)).cpu()
    return stylized_image

# Function to get files from a directory with specific extensions
def get_files(directory, extensions):
    all_files = []
    for ext in extensions:
        all_files.extend(Path(directory).rglob(f'*.{ext}'))
    return [str(file) for file in all_files]

# Automatically find image paths and model checkpoint paths
image_paths = get_files('input/images', ['jpg', 'jpeg'])  # Include other image extensions if needed
model_paths = get_files('models', ['pth'])  # Include other model extensions if used

image_selector = widgets.Dropdown(options=image_paths, description='Image:')
model_selector = widgets.Dropdown(options=model_paths, description='Model:')
run_button = widgets.Button(description='Stylize Image')

# Output widget to display results
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        print("Stylizing...")
        model = load_model(model_selector.value)
        stylized_image = stylize_image(image_selector.value, model)
        fn = image_selector.value.split("/")[-1]
        save_image(stylized_image, f"images/outputs/stylized-{fn}")
        print(f"Saved stylized-{fn} in images/outputs/")
        display(Image.open(f"images/outputs/stylized-{fn}"))

# Link button to function
run_button.on_click(on_button_clicked)

# Display widgets
display(image_selector, model_selector, run_button, output)

Dropdown(description='Image:', options=('input/images/sample1.jpg',), value='input/images/sample1.jpg')

Dropdown(description='Model:', options=('models/cuphead_10000.pth', 'models/mosaic_10000.pth', 'models/starry_…

Button(description='Stylize Image', style=ButtonStyle())

Output()

#### Video Upload Functionality

Similar to the image upload functionality, this section provides a file upload widget specifically for videos. It supports multiple video formats and allows multiple files to be uploaded simultaneously. The uploaded videos are saved in the 'input/videos' directory, which is created if it does not exist. The upload details are logged for user confirmation.


In [7]:
target_directory = 'input/videos'

# Ensure the target directory exists
if not os.path.exists(target_directory):
    os.makedirs(target_directory)
    print(f"Directory {target_directory} created.")
else:
    print(f"Target directory: {target_directory}")

# Function to handle uploaded files
def handle_upload(change):
    print("Upload started...")
    # Print the structure of 'change' to understand its content
    print(change)
    
    for file_upload in change['new']:
        print(f"Handling file: {file_upload['name']}")
        filepath = os.path.join(target_directory, file_upload['name'])
        print(f"Saving to: {filepath}")
        with open(filepath, 'wb') as f:
            f.write(file_upload['content'])
        print(f'Saved {file_upload["name"]} to {filepath}')
    # List the files in the target directory after upload
    print(f'Files in target directory ({target_directory}): {list(Path(target_directory).glob("*"))}')
    print("Upload completed.")

# Create an output widget to capture print statements
output = widgets.Output()

# Create an upload widget
upload_widget = widgets.FileUpload(accept='video/*', multiple=True)

# Function to handle the change event using output widget
def handle_upload_with_output(change):
    with output:
        handle_upload(change)

# Attach the observer to the upload widget
upload_widget.observe(handle_upload_with_output, names='value')

# Display the upload widget and output widget
display(upload_widget, output)

Target directory: input/videos


FileUpload(value=(), accept='video/*', description='Upload', multiple=True)

Output()

#### Video Processing for Style Transfer

This block sets up the functionality for processing video files for style transfer. It includes a function to retrieve video files and model checkpoints from specified directories. Users can select a video file and a model checkpoint from dropdown menus.

The `process_video` function extracts frames from the selected video, applies the style transfer to each frame using the specified model, and recompiles the frames into a new stylized video. The result is saved to the 'videos/outputs' directory and the completion is logged.


In [8]:

os.makedirs("videos/outputs", exist_ok=True)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to get video files
def get_video_files(directory, extensions=['mp4', 'gif']):
    return [str(p) for p in Path(directory).rglob('*') if p.suffix[1:] in extensions]

# Function to get model files
def get_model_files(directory, extensions=['pth']):
    return [str(p) for p in Path(directory).rglob('*') if p.suffix[1:] in extensions]

video_directory = 'input/videos'  # Path to video directory
model_directory = 'models'        # Path to model directory

videos = get_video_files(video_directory)
models = get_model_files(model_directory)

video_selector = widgets.Dropdown(options=videos, description='Select Video:')
model_selector = widgets.Dropdown(options=models, description='Select Model:')
format_selector = widgets.Dropdown(options=['avi', 'gif'], description='Output Format:')
process_button = widgets.Button(description='Process Video')
output = widgets.Output()

def process_video(b):
    with output:
        clear_output()
        video_path = video_selector.value
        model_path = model_selector.value
        output_format = format_selector.value
        transform = style_transform()  # Make sure this function is defined correctly
        model = TransformerNet().to(device)  # Make sure TransformerNet is defined correctly
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.eval()
        
        stylized_frames = []
        for frame in tqdm.tqdm(extract_frames(video_path), desc="Processing frames"):  # Ensure extract_frames is defined
            image_tensor = Variable(transform(frame)).to(device).unsqueeze(0)
            with torch.no_grad():
                stylized_image = model(image_tensor)
            stylized_frames.append(deprocess(stylized_image))  # Ensure deprocess is defined
        
        video_name = Path(video_path).stem
        output_path = f"videos/outputs/stylized-{video_name}.{output_format}"
        writer = skvideo.io.FFmpegWriter(output_path, outputdict={
            '-vcodec': 'libx264' if output_format == 'avi' else 'libx264',  # Adjust codec according to needs
            '-pix_fmt': 'yuv420p'
        })
        for frame in tqdm.tqdm(stylized_frames, desc="Writing to video"):
            writer.writeFrame(frame)
        writer.close()
        
        print(f"Video processed and saved to {output_path}")

process_button.on_click(process_video)
display(video_selector, model_selector, format_selector, process_button, output)

Dropdown(description='Select Video:', options=('input/videos/sample_video.mp4 (240p).mp4', 'input/videos/LeBro…

Dropdown(description='Select Model:', options=('models/cuphead_10000.pth', 'models/mosaic_10000.pth', 'models/…

Button(description='Process Video', style=ButtonStyle())

Output()